# Build a simple ML pipeline for image classification

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create `Pipeline` with components

**Motivations** -This tutorial shows how to train a simple deep neural network using the [Fashion MNIST dataset and Keras on Azure Machine Learning. Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.


# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
#import required libraries
from azure.ml import MLClient, dsl
from azure.ml.entities import load_component

## 1.2. Configure credential

We are using `DefaultAzureCredential` to get access to workspace. When an access token is needed, it requests one using multiple identities(`EnvironmentCredential, ManagedIdentityCredential, SharedTokenCacheCredential, VisualStudioCodeCredential, AzureCliCredential, AzurePowerShellCredential`) in turn, stopping when one provides a token.
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for more information.

`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python) for all available credentials if it does not work for you.  

In [ ]:
from azure.identity import DefaultAzureCredential

try:
    credential = DefaultAzureCredential(exclude_visual_studio_code_credential=True)
    # Check if given credential can get token successfully.
    credential.get_token('https://management.azure.com/.default')
except Exception as ex:
    # If exception happens when retrieve token, try exclude the failed credential like this then try again:
    # Exclude VSCode credential:
    # credential = DefaultAzureCredential(exclude_visual_studio_code_credential=True)
    raise Exception("Failed to retrieve a token from the included credentials due to the following exception, try to add `exclude_xxx_credential=True` to `DefaultAzureCredential` and try again.") from ex

## 1.3. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. `MLClient.from_config()` reads the file config.json and loads the details into an object named ml_client, and it expects the workspace configuration to be saved in the current directory or its parent.

In [ ]:
config_path = "./config.json"
ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

## 1.4. Retrieve or create an Azure Machine Learning compute target

In [ ]:
from azure.ml.entities import AmlCompute

# specify aml compute name.
gpu_compute_target = 'gpu-cluster'
cpu_compute_target = 'test-ci'

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print('Creating a new cpu compute target...')
    compute = AmlCompute(name=cpu_compute_target, size="STANDARD_D2_V2", min_instances=0, max_instances=4)
    ml_client.compute.begin_create_or_update(compute)

try:
    ml_client.compute.get(gpu_compute_target)
except Exception:
    print('Creating a new gpu compute target...')
    compute = AmlCompute(name=gpu_compute_target, size="STANDARD_NC6", min_instances=0, max_instances=4)
    ml_client.compute.begin_create_or_update(compute)

## 1.5. Prepare Job Input
By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. The data remains in its existing location, so no extra storage cost is incurred.

In [ ]:
from azure.ml.entities import JobInput

fashion_ds = JobInput(path="wasbs://demo@data4mldemo6150520719.blob.core.windows.net/mnist-fashion/")

In [ ]:
fashion_ds

## Configure preparation step
Using dsl component

In [52]:
%load_ext autoreload
%autoreload 2

# If you modify the componetn source code, use following code to reload it
# Otherwise notebook has cache and may not load the latest update
import importlib, prep.prep_dsl_component, train.train_dsl_component
importlib.reload(prep.prep_dsl_component)
importlib.reload(train.train_dsl_component)

# load component function from dsl component python file
from prep.prep_dsl_component import prep
from train.train_dsl_component import keras_train

# load component function from yaml
keras_score = load_component(yaml_file='./score/score.yaml')

help(prep)
help(keras_train)
help(keras_score)
print(prep())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Help on function prep in module prep.prep_dsl_component:

prep(input_data: azure.ml.dsl._types.ArtifactInput, training_data: azure.ml.dsl._types.ArtifactOutput, test_data: azure.ml.dsl._types.ArtifactOutput)

Help on function keras_train in module train.train_dsl_component:

keras_train(input_data: azure.ml.dsl._types.ArtifactInput, output_model: azure.ml.dsl._types.ArtifactOutput, epochs=10)

Help on CommandComponent in module azure.ml.entities._component.command_component object:

class CommandComponent(azure.ml.entities._component.component.Component, azure.ml.entities._job.parameterized_command.ParameterizedCommand)
 |  CommandComponent(*, name: str = None, version: str = None, description: str = None, tags: Dict = None, display_name: str = None, command: str = None, code: str = None, environment: Union[str, azure.ml.entities._assets.environment.Environment] = None, distribution: Union[azure.ml.

In [53]:
# test prep node
@dsl.pipeline(
    display_name='test-prep-step',
    description='E2E image classification pipeline with keras',
    default_compute=cpu_compute_target,
)
def image_classification_keras_minist_convnet():

    prepare_data_node = prep(input_data=fashion_ds)

    train_node = keras_train(input_data=prepare_data_node.outputs.training_data)
    train_node.compute = gpu_compute_target

    score_node = keras_score(input_data=prepare_data_node.outputs.test_data, input_model=train_node.outputs.output_model)

# create a pipeline
pipeline = image_classification_keras_minist_convnet()

In [30]:
print(pipeline.name)

elated_pea_py0r2j7zwm


In [54]:
pipeline_job = ml_client.jobs.create_or_update(pipeline, experiment_name='test-tutorials')
pipeline_job

Uploading score (0.0 MBs): 100%|##########| 4633/4633 [00:01<00:00, 3855.24it/s]




Experiment,Name,Type,Status,Details Page
test-tutorials,careful_spinach_l9ntzmp2gr,pipeline,Preparing,Link to Azure Machine Learning studio


# 2. Define command component via YAML
Below is a basic example to define command component  using YAML.


In [ ]:
prepare_dataset = load_component(yaml_file='./keras-mnist-fashion/prepare.yaml')
train_model = load_component(yaml_file='./keras-mnist-fashion/train.yaml')

# 3. Basic pipeline job

## 3.1 Build pipeline

In step one, we will load the image and labels from Fashion MNIST dataset into mnist_train.csv and mnist_test.csv. Then in step two, train CNN with Keras

In [ ]:
# define pipeline
@dsl.pipeline(
    display_name='image_classification_keras_minist_convnet',
    description='E2E image classification pipeline with keras',
    default_compute=cpu_compute_target,
)
def image_classification_keras_minist_convnet():
    prepare_dataset_node = prepare_dataset(input_path=fashion_ds)

    train_node = (input_path=prepare_dataset_node.outputs.output_path)
    train_node.compute = gpu_compute_target
    
    # Return: pipeline outputs
    return {
        "trained_model": train_node.outputs.output_model,
    }

# create a pipeline
pipeline = image_classification_keras_minist_convnet()

In [ ]:
print(pipeline)

# 3.2 Submit pipeline job

In [ ]:
pipeline_job = ml_client.jobs.create_or_update(pipeline, experiment_name='pipeline_samples')
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](/sdk/jobs/pipelines/)